In [1]:
#4bit量子化 & deepspeedができるかテスト

#これ?
#https://github.com/git-cloner/llama-lora-fine-tuning/blob/main/fastchat/train/train_lora.py

In [2]:

from transformers import (AutoModelForCausalLM, AutoTokenizer,
                           pipeline,
                           TrainingArguments,DataCollatorForLanguageModeling,
                           Trainer,AutoConfig,
                            )

from peft import LoraConfig, get_peft_model
from transformers.deepspeed import HfDeepSpeedConfig
import torch
import deepspeed
from datasets import Dataset
model_name = "meta-llama/Llama-2-7b-chat-hf"

/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-10-05 01:35:08,341] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/user/miniconda3/envs/ft/lib/python3.11/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_config = AutoConfig.from_pretrained(model_name)

text="""
私はその人を常に先生と呼んでいた。だからここでもただ先生と書くだけで本名は打ち明けない。これは世間を憚かる遠慮というよりも、その方が私にとって自然だからである。私はその人の記憶を呼び起すごとに、すぐ「先生」といいたくなる。筆を執っても心持は同じ事である。よそよそしい頭文字などはとても使う気にならない。
私が先生と知り合いになったのは鎌倉である。その時私はまだ若々しい書生であった。暑中休暇を利用して海水浴に行った友達からぜひ来いという端書を受け取ったので、私は多少の金を工面して、出掛ける事にした。私は金の工面に二、三日を費やした。ところが私が鎌倉に着いて三日と経たないうちに、私を呼び寄せた友達は、急に国元から帰れという電報を受け取った。電報には母が病気だからと断ってあったけれども友達はそれを信じなかった。友達はかねてから国元にいる親たちに勧まない結婚を強いられていた。彼は現代の習慣からいうと結婚するにはあまり年が若過ぎた。それに肝心の当人が気に入らなかった。それで夏休みに当然帰るべきところを、わざと避けて東京の近くで遊んでいたのである。彼は電報を私に見せてどうしようと相談をした。私にはどうしていいか分らなかった。けれども実際彼の母が病気であるとすれば彼は固より帰るべきはずであった。それで彼はとうとう帰る事になった。せっかく来た私は一人取り残された。
"""
text_list=text.strip().split("｡")
data_list = [{"text": i} for i in text_list]

# tokenize
dataset = Dataset.from_dict({"text": [item["text"] for item in data_list]})
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(lambda examples: tokenizer(examples['text'],
                                                           truncation=True,
                                                           padding='max_length',
                                                           max_length=100
                                                           # max_length=10
                                                           ),
                            )

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Map: 100%|██████████| 1/1 [00:00<00:00, 125.12 examples/s]


In [4]:

ds_config=  {
        "stage": 3,
        "offload_param": {
            "device": "cpu"
            },
        #"bf16":{
        #    "enabled":"auto"
        "fp16":{
            "enabled":"auto"
        }       #}
        }


world_size=1
hidden_size = model_config.hidden_size
ds_config["train_batch_size"] = 1 * world_size
ds_config["train_micro_batch_size_per_gpu"] = 1
ds_config["reduce_bucket_size"] = hidden_size*hidden_size
ds_config["stage3_prefetch_bucket_size"] = 0.9 * hidden_size * hidden_size
ds_config["stage3_param_persistence_threshold"] = 10 * hidden_size

dschf = HfDeepSpeedConfig(ds_config)  # zero3を使用するために必要(モデルロード前に実行する必要がある)
model = AutoModelForCausalLM.from_pretrained(model_name)

r=8
peft_config = LoraConfig(
    task_type="CAUSAL_LM", inference_mode=False, r=r, lora_alpha=32,
    lora_dropout=0.1,
    target_modules=None,
)
model = get_peft_model(model, peft_config)


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.54it/s]


In [5]:
ds_engine = deepspeed.initialize(model=model, 
                                 config_params=ds_config)[0]
ds_model = ds_engine.module  # .eval()

[2023-10-05 01:35:48,690] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.10.3, git-hash=unknown, git-branch=unknown
[2023-10-05 01:35:48,691] [INFO] [comm.py:637:init_distributed] cdb=None
[2023-10-05 01:35:48,691] [INFO] [comm.py:652:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2023-10-05 01:35:48,743] [INFO] [comm.py:702:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.11.2, master_port=29500
[2023-10-05 01:35:48,743] [INFO] [comm.py:668:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl
[2023-10-05 01:35:52,094] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2023-10-05 01:35:52,096] [INFO] [config.py:967:print] DeepSpeedEngine configuration:
[2023-10-05 01:35:52,097] [INFO] [config.py:971:print]   activation_checkpointing_config  {
    "partition_activations": false, 
    "contiguous_memory_opt

In [6]:
#for name, param in ds_model.named_parameters():
#    print(name,param.requires_grad)

In [9]:
epochs=1
training_args = TrainingArguments(
    output_dir='./outputs',
    num_train_epochs=epochs,  # エポック数
    # per_device_train_batch_size=1,  # バッチサイズ
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,  # 勾配チェックポイント
    #fp16=True,  # fp16
    optim='adafactor',  # オプティマイザの種類
    deepspeed='../llm/deepspeed/4bit_train.json',  # deepspeedのconfigへのpath
    logging_steps=100,  # 途中経過を表示する間隔
)

In [11]:

trainer = Trainer(
    model=ds_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)


In [ ]:


# %%
result = trainer.train()

In [ ]:

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, 
            )